# Reduce Concussion Risk on Punt Plays by Penalizing All Blindside Blocks

## Abstract

This notebook uses National Football League (NFL) Next Gen Stats (NGS) data and game video footage to categorize punt plays that resulted in a concussion. The objective of this notebook is to provide a recommendation and evidence for changing the [Official Playing Rules of the National Football League](https://operations.nfl.com/media/3277/2018-nfl-rulebook_final-version.pdf) (hereafter "Rulebook") to reduce the risk of concussions on punt plays while maintaining the integrity of the game of football. Through cluster analysis, review of video footage, and review of the Rulebook, I find that the most promising rule change is to penalize all blindside blocks. Currently, players moving away from their own end line are exempt from a penalty for a blindside block. Blindside blocks by a player moving away from their own end line characterize the most common pattern of concussion events on the punt plays in the data. While I identify other patterns of concussion events that offer other opportunities to reduce concussion risk, penalizing all blindside blocks is the most clearly evidenced, would require the simplest rule change, and would best preserve the integrity of the game.

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import os
from sklearn.cluster import DBSCAN, AffinityPropagation
from sklearn.preprocessing import Normalizer
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

## Data Preparation

37 punt plays resulted in a concussion in the 2016 and 2017 NFL seasons, including the preseason and playoffs. The NFL provided video review data of each such play. For 33 of the plays, the "primary partner" to the helmet impact determined to have caused the concussion is identified.

In [ ]:
# Import and view concussion play data
video_review = pd.read_csv('../input/video_review.csv')
video_review

The NFL provided NGS data for all plays in the 2016 and 2017 seasons. I recover the NGS data for punt plays that resulted in a concussion and merge those data with the video review data. At every 0.1-second time interval since the start of the play, the NGS data provide the x and y coordinates, distance traveled since last time point, and angles of orientation and movement for the player that experienced the concussion and his primary partner.

In [ ]:
# Retrieve NGS data for concussion plays
NGS = []
for f in os.listdir('../input'):
    if f.startswith('NGS'):
        df = pd.read_csv('../input/' + f)
        df = df[(df['GameKey'].isin(list(video_review['GameKey']))) & (df['PlayID'].isin(list(video_review['PlayID'])))]
        NGS.append(df)
NGS = pd.concat(NGS)
del NGS['Season_Year'], NGS['Event']

In [ ]:
# Merge with NGS data
filtered_NGS = video_review[['GameKey', 'PlayID', 'GSISID', 'Primary_Partner_GSISID']].merge(NGS, how='left', on=['GameKey', 'PlayID', 'GSISID'])
NGS['Primary_Partner_GSISID'] = NGS['GSISID']
filtered_NGS['Primary_Partner_GSISID'] = pd.to_numeric(filtered_NGS['Primary_Partner_GSISID'], errors='coerce')
filtered_NGS = filtered_NGS.merge(NGS.drop(['GSISID'], axis=1), how='left', on=['GameKey', 'PlayID', 'Primary_Partner_GSISID', 'Time'], suffixes=('', '_Partner'))

The data do not identify the time of the impact determined to have caused the concussion. I infer the time of impact by calculating the distance between the player that experienced the concussion and his partner and finding the earliest time where that distance is minimized. I also calculate the difference in angle of orientation and difference in angle of movement between the two players.

In [ ]:
# Create features
filtered_NGS['PlayKey'] = filtered_NGS['GameKey'] + filtered_NGS['PlayID'] + filtered_NGS['GSISID']
filtered_NGS['player_dist'] = ((filtered_NGS['x'] - filtered_NGS['x_Partner'])**2 + (filtered_NGS['y'] - filtered_NGS['y_Partner'])**2)**0.5
filtered_NGS['Time'] = pd.to_datetime(filtered_NGS['Time'])
filtered_NGS['time_since_play_start'] = (filtered_NGS['Time'] - filtered_NGS.groupby(['PlayKey'])['Time'].transform(min)).dt.total_seconds()
filtered_NGS['impact_time'] = np.where(filtered_NGS.groupby(['PlayKey'])['player_dist'].transform(min) == filtered_NGS['player_dist'], filtered_NGS['time_since_play_start'], np.nan)
filtered_NGS['time_to_impact'] = filtered_NGS.groupby(['PlayKey'])['impact_time'].transform(min) - filtered_NGS['time_since_play_start']
filtered_NGS['o_diff'] = np.min([filtered_NGS['o'] - filtered_NGS['o_Partner'], 180 - (filtered_NGS['o'] - filtered_NGS['o_Partner'])], axis=0)
filtered_NGS['dir_diff'] = np.min([filtered_NGS['dir'] - filtered_NGS['dir_Partner'], 180 - (filtered_NGS['dir'] - filtered_NGS['dir_Partner'])], axis=0)

I plan to categorize plays based on the players' speeds and angle differences in the half-second leading up to the inferred time of impact (note that distance traveled over 0.1 seconds is a measure of speed). I restrict the data to the relevant timeframe and features and reshape the data so that each play is represented by a single row of feature values over time.

In [ ]:
# Query model input
model_input = filtered_NGS[(0.5 >= filtered_NGS['time_to_impact']) & (filtered_NGS['time_to_impact'] >= 0)]
model_input = model_input.sort_values(['PlayKey', 'time_to_impact'])
model_input['obs_num'] = model_input.groupby(['PlayKey']).cumcount()
model_input = model_input.pivot(index='PlayKey', columns='obs_num', values=['o_diff', 'dir_diff', 'dis', 'dis_Partner'])

## Analysis

I use the Affinity Propagation clustering algorithm, [implemented by the scikit-learn library](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html), to categorize plays. Compared to other clustering methods, Affinity Propagation has [the advantages of data-driven determination of the number of clusters, absence of random elements, and particular suitability to small data](https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation). I [normalize the data](https://datascience.stackexchange.com/questions/6715/is-it-necessary-to-standardize-your-data-before-clustering) so that the algorithm assigns similar weight to each feature.

The algorithm identifies six clusters. The largest cluster has ten plays and the smallest has one.

In [ ]:
# Perform clustering
model = AffinityPropagation().fit(Normalizer().fit_transform(model_input))
labels = pd.Series(model.labels_, name= 'cluster', index=model_input.index)
print('Number of plays in each cluster:')
labels.groupby(labels).count().sort_values(ascending=False)

To characterize the clusters, we can view the video footage of plays in each cluster. While we should not expect to find a single concept to characterize every video in any given cluster, we are able to find informative patterns in each cluster with multiple videos. We are also able to identify these patterns in plays that were not clustered because they did not have an identified primary partner.

Most of the plays in the largest cluster are characterized by a blindside block on a kicking team player as he turns upfield. The most obvious examples follow. Note that this event is hazardous for both players; while the first two videos below show a concussion experienced by the blocked player, the blocking player (#33) experienced a concussion in the third video.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153280/Wing_37_yard_punt-cPHvctKg-20181119_165941654_5000k.mp4" type="video/mp4"></video>')

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153258/61_yard_Punt_by_Brett_Kern-g8sqyGTz-20181119_162413664_5000k.mp4" type="video/mp4"></video>')

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153272/Haack_42_yard_punt-iP6aZSRU-20181119_165050694_5000k.mp4" type="video/mp4"></video>')

One of the four videos without a recorded primary partner fits the same characterization as the above videos. Notice how 11 seconds into the video #94 on the receiving team pulls up and avoids making a blindside block (yet nullifies the opponent) unlike his teammate #55 two seconds later.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153321/Lechler_55_yd_punt-lG1K51rf-20181119_173634665_5000k.mp4" type="video/mp4"></video>')

Most of the plays in the second-largest cluster are characterized by the receiver not cleanly fielding the punt, as shown in the following videos.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153244/Punt_by_Brad_Wing-5hmlbMBx-20181119_155243111_5000k.mp4" type="video/mp4"></video>')

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153235/Punt_Return_by_Jamison_Crowder-HnVouDMH-20181119_153041501_5000k.mp4" type="video/mp4"></video>')

In [ ]:
HTML('<video width="800" height="600" controls> <source src="https://nfl-vod.cdn.anvato.net/league/5691/18/11/25/284956/284956_12D27120C06E4DB994040750FB43991D_181125_284956_way_punt_3200.mp4" type="video/mp4"></video>')

One of the four plays without a recorded primary partner fits the same characterization as the above videos.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153274/Haack_punts_41_yards-SRJMeOc3-20181119_165546590_5000k.mp4" type="video/mp4"></video>')

One of the plays in the second-largest cluster, despite being characterized by a receiver not cleanly fielding a punt, appears to instead belong in the largest cluster. 12 seconds into the video a player on the receiving team clearly levels an opposing player as the opponent turns upfield.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="https://nfl-vod.cdn.anvato.net/league/5691/18/11/25/284954/284954_75F12432BA90408C92660A696C1A12C8_181125_284954_huber_punt_3200.mp4" type="video/mp4"></video>')

The second-largest cluster also contains the two plays where the concussion occurs behind the line of scrimmage. Both concussions are experienced by a fallen left wing.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153326/Sanchez_27_yd_punt-r51JAWPm-20181119_174359780_5000k.mp4" type="video/mp4"></video>')

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153245/Punt_by_Brad_Nortman-QiQqjFdU-20181119_155917392_5000k.mp4" type="video/mp4"></video>')

Most of the plays in the third-largest cluster are characterized by friendly fire. No other cluster has more than one friendly fire play.

In [ ]:
video_review['PlayKey'] = video_review['GameKey'] + video_review['PlayID'] + video_review['GSISID']
video_review = video_review.merge(labels.reset_index(), how='left', on='PlayKey').sort_values('cluster')
video_review.groupby(['cluster'])['Friendly_Fire'].value_counts()

The most obvious friendly fire plays occur when two tacklers converge on the ball handler from opposite sides and collide.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153273/King_62_yard_punt-BSOws7nQ-20181119_165306255_5000k.mp4" type="video/mp4"></video>')

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153257/40_yard_Punt_by_Brad_Nortman-oSbtDlHu-20181119_162303930_5000k.mp4" type="video/mp4"></video>')

Two plays in the third-largest cluster not labeled as friendly fire appear to be friendly fire regardless.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153247/Punt_by_Tress_Way-QsI21aYF-20181119_160141260_5000k.mp4" type="video/mp4"></video>')

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153260/Bolden_runs_into_Colquitt_during_punt-LpPhshZz-20181119_162648901_5000k.mp4" type="video/mp4"></video>')

The only play in the third-largest cluster not labeled as or appearing to be friendly fire is characterized by the receiver not catching a punt, and so appears to belong in the second-largest cluster.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153246/Punt_by_Brad_Nortman_2-fbS6OgDd-20181119_160019423_5000k.mp4" type="video/mp4"></video>')

Most of the plays in the fourth-largest cluster are characterized by a player pushing an opponent into a third player.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153242/Punt_by_Dustin_Colquitt-joFpAUDf-20181119_155010468_5000k.mp4"></video>')

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153240/Punt_by_Thomas_Morstead-eZpDKgMR-20181119_154525222_5000k.mp4" type="video/mp4"></video>')

One of the plays in the fourth-largest cluster is characterized by two tacklers converging on the receiver from opposite sides and colliding, and so appears to belong in the third-largest cluster.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153236/Punt_by_Brad_Wing-SMRxqgb2-20181119_153645589_5000k.mp4" type="video/mp4"></video>')

The fifth-largest cluster is characterized by unusual plays, including a fake punt and a player accidentally kneeing his teammate in the back of the head after the play was over.

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153248/Rush_by_Jon_Ryan-Csg9PS77-20181119_160437472_5000k.mp4" type="video/mp4"></video>')

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153291/Palardy_53_yard_punt-XTESVMq9-20181119_170509550_5000k.mp4" type="video/mp4"></video>')

The only other play in the fifth-largest cluster appears to belong in the largest cluster, as it involves a blindside block on a kicking team player (though he did not fall in this case).

In [ ]:
HTML('<video width="800" height="600" controls> <source src="http://a.video.nfl.com//films/vodzilla/153252/44_yard_Punt_by_Justin_Vogel-n7U6IS6I-20181119_161556468_5000k.mp4" type="video/mp4"></video>')

## Recap and Recommendation

Through a combination of cluster analysis and video review of individual plays, I find that most plays are characterized by at least one of the following:
* A blindside block on a kicking team player as he turns upfield
* Failure to catch a punt
* A wing falling while attempting to block
* Friendly fire, including when two tacklers converge on the receiver from opposite sides and collide; and
* A player pushing an opponent into a third player

These characterizations indicate multiple options for the NFL to change its rules to reduce concussion risk on punt plays. One rule change stands out due to its clarity of evidence, its simplicity, and its keeping of the integrity of the game of football: **penalize all blindside blocks**.

Rule 12-2-7-a-10 of the 2018 Rulebook states that "A player who receives a 'blindside' block when the path of the blocker is toward or parallel to his own end line" is in a defenseless posture. "Forcibly hitting the defenseless player’s head or neck area with the helmet, facemask, forearm, or shoulder" incurs a penalty of 15 yards and, if judged by the officials to be flagrant, disqualification of the offending player. However, the blindside blocks associated with concussions on punt plays occur when the path of the blocker is *away* from his own end line. The current rules do not offer defenseless posture protection in this situation. I recommend that the NFL remove the text "when the path of the blocker is toward or parallel to his own end line" from rule 12-2-7-a-10. I would expect this change to reduce concussion risk not only on punt plays but on all plays that involve change of possession far ahead of the line of scrimmage, such as kickoffs and interception returns.

I also recommend that the NFL define "blindside block" in the Rulebook. The National Federation of State High School Associations defined a blindside block as [“a block against an opponent other than the runner, who does not see the blocker approaching.”](https://www.nfhs.org/articles/new-blocking-kicking-rules-address-risk-minimization-in-high-school-football/) This definition would be consistent with the intent of the recommended rule change.

Another rule change option would be to treat a muffed or otherwise not caught punt as a dead ball when it first strikes the ground. This rule change would allow a receiver to vacate the landing area of a punt for his own safety without fearing that the kicking team will worsen his team's field position by downing the ball after it bounces closer to his own end line. It would also stop a receiver from advancing the ball after recovering it from the ground. Recovering a ball from the ground causes the receiver to take his focus off the oncoming defenders and may surprise defenders, leading to awkward movement and increased risk of dangerous and undesired collisions. However, I do not recommend this option as it would significantly impact the integrity of the game of football. Many punts would end with the strange sight of the players already leaving the field as the ball strikes the ground. Teams may adapt to the lack of field position risk by rushing punts with all eleven players. This rule change would also require extensive revision of the Rulebook to make all parts of the Rulebook consistent, including the sections on and other mentions of "fair catch" and "first touching."

The remaining characterizations listed above are also difficult to address in an efficacious manner without threatening the integrity of the game of football. Blockers falling near the line of scrimmage is not a punt-specific problem and characterized only one concussion per season. Any change to prevent concussions of that nature would likely impact football strategy and tactics on almost all plays. Friendly fire is practically impossible to prevent through officiating (a penalty cannot be assessed for actions against a teammate) and tends to occur in the non-punt-specific scenario of tacklers converging on the ball handler. Inhibiting the ability of players to push opponents would also signficantly impact the integrity of the game: rule 12-1-2-c explicitly allows that "A kicking team player may use his hands/arms to ward off or to push or pull a receiver who is legally or illegally attempting to obstruct his attempt to proceed."